In [1]:
import os
import re
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit.Chem import rdMolDescriptors as rdMD
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
TARGETS = ["Tg", "FFV", "Tc", "Density", "Rg"] 

def get_data_paths():
    BASE_PATH = Path(os.getenv('NEURIPS_DATA_PATH', 'kaggle/input/neurips-open-polymer-prediction-2025'))
    EXTRA_BASE = Path(os.getenv('EXTRA_DATA_BASE', 'kaggle/input/smiles-extra-data'))
    TC_BASE = Path(os.getenv('TC_DATA_BASE', 'kaggle/input/tc-smiles'))

    paths = {
        'train_csv': Path(os.getenv('TRAIN_CSV_PATH', BASE_PATH / 'train.csv')),
        'test_csv': Path(os.getenv('TEST_CSV_PATH', BASE_PATH / 'test.csv')),
        'sample_submission': Path(os.getenv('SAMPLE_SUBMISSION_PATH', BASE_PATH / 'sample_submission.csv')),

        'tc_data': Path(os.getenv('TC_DATA_PATH', TC_BASE / 'Tc_SMILES.csv')),
        'tg_jcim_data': Path(os.getenv('TG_JCIM_PATH', EXTRA_BASE / 'JCIM_sup_bigsmiles.csv')),
        'tg_excel_data': Path(os.getenv('TG_EXCEL_PATH', EXTRA_BASE / 'data_tg3.xlsx')),
        'density_data': Path(os.getenv('DENSITY_PATH', EXTRA_BASE / 'data_dnst1.xlsx')),
        'ffv_data': Path(os.getenv('FFV_DATA_PATH', BASE_PATH / 'train_supplement' / 'dataset4.csv')),
        'dataset1': Path(os.getenv('DATASET1_PATH', BASE_PATH / 'train_supplement' / 'dataset1.csv')),
        'dataset2': Path(os.getenv('DATASET2_PATH', BASE_PATH / 'train_supplement' / 'dataset2.csv')),
        'dataset3': Path(os.getenv('DATASET3_PATH', BASE_PATH / 'train_supplement' / 'dataset3.csv')),
    }
    return paths

P = get_data_paths()
for k, v in P.items():
    print(f"{k}: {v}  {'[OK]' if v.exists() else '[MISSING]'}")

# 核心官方数据
train = pd.read_csv(P['train_csv'])
test  = pd.read_csv(P['test_csv'])
sub   = pd.read_csv(P['sample_submission'])
print("Train/Test/Sub:", train.shape, test.shape, sub.shape)

train_csv: kaggle/input/neurips-open-polymer-prediction-2025/train.csv  [OK]
test_csv: kaggle/input/neurips-open-polymer-prediction-2025/test.csv  [OK]
sample_submission: kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv  [OK]
tc_data: kaggle/input/tc-smiles/Tc_SMILES.csv  [OK]
tg_jcim_data: kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv  [OK]
tg_excel_data: kaggle/input/smiles-extra-data/data_tg3.xlsx  [OK]
density_data: kaggle/input/smiles-extra-data/data_dnst1.xlsx  [OK]
ffv_data: kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv  [OK]
dataset1: kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv  [OK]
dataset2: kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv  [OK]
dataset3: kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv  [OK]
Train/Test/Sub: (7973, 7) (3, 2) (3, 6)


In [2]:
def add_extra_data(df_train: pd.DataFrame, df_extra: pd.DataFrame, target: str, source_name: str = "extra") -> pd.DataFrame:
    """
    将外部数据 merge 到 train：
    - 如果 train 里有相同 SMILES 且 target 缺失，用 extra 填补
    - 如果 extra 里有新 SMILES，追加到 train
    - 对同一个 SMILES target 出现多个值时取均值
    - 打印出重复 SMILES 的数量 & 差异
    """
    print(f"[INFO] Working on {source_name} (target={target})")
    before = len(df_train)

    # 只保留 SMILES + target
    df_extra = df_extra[['SMILES', target]].dropna(subset=[target]).copy()

    # 聚合外部数据，避免重复
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()

    # 检查重复
    common = set(df_train['SMILES']) & set(df_extra['SMILES'])
    if common:
        merged_common = pd.merge(
            df_train[df_train['SMILES'].isin(common)][['SMILES', target]],
            df_extra[df_extra['SMILES'].isin(common)],
            on='SMILES',
            suffixes=('_train', '_extra')
        )
        diffs = (merged_common[f"{target}_train"] - merged_common[f"{target}_extra"]).abs()
        print(f"  重复 SMILES: {len(merged_common)}, 其中 {sum(diffs>1e-6)} 条数值不同 (mean diff={diffs.mean():.3f})")

    # merge
    df_train = df_train.merge(df_extra, on='SMILES', how='outer', suffixes=('', '_extra'))

    # 填补缺失
    mask = df_train[target].isna() & df_train[f"{target}_extra"].notna()
    df_train.loc[mask, target] = df_train.loc[mask, f"{target}_extra"]

    # 删除多余列
    if f"{target}_extra" in df_train.columns:
        df_train = df_train.drop(columns=[f"{target}_extra"])

    after = len(df_train)
    print(f"  [{target}] 增强: {after - before:+d} 条新样本, 填补 {mask.sum()} 条缺失")

    return df_train


In [3]:
paths = get_data_paths()

if os.path.exists(paths['tc_data']):       
    train = add_extra_data(
        train,
        pd.read_csv(paths['tc_data']).rename(columns={'TC_mean':'Tc'}),
        target='Tc',
        source_name='tc_data'
    )

if os.path.exists(paths['tg_jcim_data']):  
    train = add_extra_data(
        train,
        pd.read_csv(paths['tg_jcim_data'], usecols=['SMILES','Tg (C)']).rename(columns={'Tg (C)':'Tg'}),
        target='Tg',
        source_name='tg_jcim'
    )

if os.path.exists(paths['tg_excel_data']):
    train = add_extra_data(
        train,
        pd.read_excel(paths['tg_excel_data']).rename(columns={'Tg [K]':'Tg'}).assign(Tg=lambda df: df['Tg'] - 273.15),
        target='Tg',
        source_name='tg_excel_K_to_C'
    )

if os.path.exists(paths['dataset3']):
    train = add_extra_data(
        train,
        pd.read_csv(paths['dataset3']),  # 已是列名 SMILES, Tg
        target='Tg',
        source_name='dataset3'
    )

if os.path.exists(paths['density_data']):
    train = add_extra_data(
        train,
        pd.read_excel(paths['density_data'])
          .rename(columns={'density(g/cm3)':'Density'})
          .assign(Density=lambda df: pd.to_numeric(df['Density'], errors='coerce') - 0.118),
        target='Density',
        source_name='density_extra_minus_0p118'
    )

if os.path.exists(paths['dataset1']):
    train = add_extra_data(
        train,
        pd.read_csv(paths['dataset1']).rename(columns={'TC_mean':'Tc'}),
        target='Tc',
        source_name='dataset1'
    )

if os.path.exists(paths['ffv_data']):
    train = add_extra_data(
        train,
        pd.read_csv(paths['ffv_data']).rename(columns={'FFV':'FFV'}),
        target='FFV',
        source_name='ffv_dataset4'
    )

print(f"train: {train.shape}")
print("targets non-null:", {t: int(train[t].notna().sum()) for t in TARGETS if t in train.columns})

[INFO] Working on tc_data (target=Tc)
  重复 SMILES: 737, 其中 2 条数值不同 (mean diff=0.000)
  [Tc] 增强: +130 条新样本, 填补 130 条缺失
[INFO] Working on tg_jcim (target=Tg)
  重复 SMILES: 7, 其中 0 条数值不同 (mean diff=0.000)
  [Tg] 增强: +655 条新样本, 填补 655 条缺失
[INFO] Working on tg_excel_K_to_C (target=Tg)
  [Tg] 增强: +499 条新样本, 填补 499 条缺失
[INFO] Working on dataset3 (target=Tg)
  [Tg] 增强: +46 条新样本, 填补 46 条缺失
[INFO] Working on density_extra_minus_0p118 (target=Density)
  重复 SMILES: 4, 其中 2 条数值不同 (mean diff=0.055)
  [Density] 增强: +782 条新样本, 填补 784 条缺失
[INFO] Working on dataset1 (target=Tc)
  重复 SMILES: 867, 其中 2 条数值不同 (mean diff=0.000)
  [Tc] 增强: +0 条新样本, 填补 0 条缺失
[INFO] Working on ffv_dataset4 (target=FFV)
  重复 SMILES: 37, 其中 0 条数值不同 (mean diff=nan)
  [FFV] 增强: +825 条新样本, 填补 862 条缺失
train: (10910, 7)
targets non-null: {'Tg': 1711, 'FFV': 7892, 'Tc': 867, 'Density': 1397, 'Rg': 614}


In [4]:
# Check for missing values
print("\nMissing values in train data:")
print(train.isnull().sum())


Missing values in train data:
id          2937
SMILES         0
Tg          9199
FFV         3018
Tc         10043
Density     9513
Rg         10296
dtype: int64


In [5]:
def compute_all_smiles_features(smiles: str) -> dict:
    """提取 SMILES 字符串的字符/化学符号/占位符特征（不依赖 RDKit）"""
    if not isinstance(smiles, str):
        smiles = str(smiles)
    feats = {}

    # 基础统计
    feats['smiles_length'] = len(smiles)
    feats['capital_letters'] = sum(c.isupper() for c in smiles)
    feats['lowercase_letters'] = sum(c.islower() for c in smiles)
    feats['digits'] = sum(c.isdigit() for c in smiles)

    # 符号统计
    feats['parentheses'] = smiles.count('(') + smiles.count(')')
    feats['brackets'] = smiles.count('[') + smiles.count(']')
    feats['braces'] = smiles.count('{') + smiles.count('}')
    feats['equals'] = smiles.count('=')
    feats['hashes'] = smiles.count('#')
    feats['colons'] = smiles.count(':')
    feats['ats'] = smiles.count('@')
    feats['slashes'] = smiles.count('/') + smiles.count('\\')
    feats['plus_minus'] = smiles.count('+') + smiles.count('-')

    # 元素计数
    feats['C_count'] = smiles.count('C') + smiles.count('c')
    feats['O_count'] = smiles.count('O') + smiles.count('o')
    feats['N_count'] = smiles.count('N') + smiles.count('n')
    feats['S_count'] = smiles.count('S') + smiles.count('s')
    feats['P_count'] = smiles.count('P') + smiles.count('p')
    feats['F_count'] = smiles.count('F') + smiles.count('f')
    feats['Cl_count'] = smiles.count('Cl') + smiles.count('cl')
    feats['Br_count'] = smiles.count('Br') + smiles.count('br')
    feats['I_count'] = smiles.count('I') + smiles.count('i')

    # 结构模式
    feats['has_ring'] = int(any(d in smiles for d in '123456789'))
    feats['has_double_bond'] = int('=' in smiles)
    feats['has_triple_bond'] = int('#' in smiles)
    feats['has_aromatic'] = int(any(c in smiles for c in 'cnos'))

    # 元素比例
    feats['O_to_C_ratio'] = feats['O_count'] / (feats['C_count'] + 1e-5)
    feats['N_to_C_ratio'] = feats['N_count'] / (feats['C_count'] + 1e-5)
    feats['heteroatom_ratio'] = (
        feats['O_count'] + feats['N_count'] + feats['S_count'] + feats['P_count']
    ) / (feats['C_count'] + 1e-5)

    # 占位符特征
    feats['star_count'] = smiles.count('*')
    feats['R_placeholder_count'] = len(re.findall(r"\[R[0-9']*\]", smiles))
    feats['any_placeholder'] = int((feats['star_count'] > 0) or (feats['R_placeholder_count'] > 0))

    return feats


def add_all_features(df: pd.DataFrame, col: str = 'SMILES') -> pd.DataFrame:
    """给 DataFrame 添加所有 SMILES 特征"""
    feats = df[col].astype(str).apply(compute_all_smiles_features).apply(pd.Series)
    feats.index = df.index
    return pd.concat([df, feats], axis=1)

In [6]:
train_with_feats = add_all_features(train, col='SMILES')
test_with_feats  = add_all_features(test,  col='SMILES')

print("Train with features:", train_with_feats.shape)
print("Test with features:", test_with_feats.shape)

Train with features: (10910, 39)
Test with features: (3, 34)


In [7]:
print(train_with_feats.head(3))

            id                                 SMILES          Tg  FFV     Tc  \
0          NaN          */C(=C(/*)\c1ccccc1)/c1ccccc1  206.569886  NaN    NaN   
1  218059466.0            */C(=C(/*)c1ccccc1)c1ccccc1  206.569886  NaN    NaN   
2          NaN  */C(=C(\c1ccccc1)c1ccc(*)cc1)c1ccccc1         NaN  NaN  0.338   

   Density  Rg  smiles_length  capital_letters  lowercase_letters  ...  \
0      NaN NaN           29.0              2.0               12.0  ...   
1      NaN NaN           27.0              2.0               12.0  ...   
2      NaN NaN           37.0              2.0               18.0  ...   

   has_ring  has_double_bond  has_triple_bond  has_aromatic  O_to_C_ratio  \
0       1.0              1.0              0.0           1.0           0.0   
1       1.0              1.0              0.0           1.0           0.0   
2       1.0              1.0              0.0           1.0           0.0   

   N_to_C_ratio  heteroatom_ratio  star_count  R_placeholder_count  \

In [8]:
def handle_missing_values(df, exclude_cols=None):
    """
    只对特征列做缺失值填充，排除 id / SMILES / targets 等
    """
    df = df.copy()
    if exclude_cols is None:
        exclude_cols = []

    # 找出需要处理的数值列
    numeric_cols = [
        c for c in df.select_dtypes(include=[np.number]).columns
        if c not in exclude_cols
    ]

    for col in numeric_cols:
        if df[col].isnull().any():
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)

    return df

In [9]:
EXCLUDE_BASE = ['id', 'SMILES'] + TARGETS

train_df = handle_missing_values(train_with_feats, exclude_cols=EXCLUDE_BASE)
test_df  = handle_missing_values(test_with_feats, exclude_cols=EXCLUDE_BASE)

# Check for missing values
print("\nMissing values in train data:")
train_df = train_df.drop(columns=['id'])
print(f'train_df.shape:{train_df.shape}')
print(train_df.isnull().sum())


Missing values in train data:
train_df.shape:(10910, 38)
SMILES                     0
Tg                      9199
FFV                     3018
Tc                     10043
Density                 9513
Rg                     10296
smiles_length              0
capital_letters            0
lowercase_letters          0
digits                     0
parentheses                0
brackets                   0
braces                     0
equals                     0
hashes                     0
colons                     0
ats                        0
slashes                    0
plus_minus                 0
C_count                    0
O_count                    0
N_count                    0
S_count                    0
P_count                    0
F_count                    0
Cl_count                   0
Br_count                   0
I_count                    0
has_ring                   0
has_double_bond            0
has_triple_bond            0
has_aromatic               0
O_to_C_ratio  

In [32]:
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.error')  # 关掉 RDKit 的报错输出

def _mol_from_smiles_robust(smi: str):
    """更稳的解析：失败时尝试 sanitize=False 再手动 Sanitize。"""
    if not isinstance(smi, str) or not smi.strip():
        return None
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        return mol
    mol = Chem.MolFromSmiles(smi, sanitize=False)
    if mol is None:
        return None
    try:
        Chem.SanitizeMol(mol)
        return mol
    except Exception:
        return None

def is_valid_smiles(smi: str) -> bool:
    """True 表示 RDKit 能成功解析。"""
    return _mol_from_smiles_robust(smi) is not None

def drop_invalid_smiles(df, smiles_col: str = "SMILES", show_examples: int = 5):
    """
    删除 RDKit 无法解析的 SMILES 行，并打印前后差异。
    返回：clean_df（已删除无效行，索引已重置）
    """
    df = df.copy()
    n_before = len(df)

    valid_mask = df[smiles_col].apply(is_valid_smiles)
    n_invalid = int((~valid_mask).sum())
    pct_invalid = n_invalid / n_before * 100 if n_before > 0 else 0.0

    # 打印摘要
    print(f"[SMILES 校验] 总计: {n_before}  | 无效: {n_invalid} ({pct_invalid:.3f}%)")
    if n_invalid > 0:
        bad_examples = df.loc[~valid_mask, smiles_col].dropna().astype(str).unique()[:show_examples]
        print("无效样例（最多展示几条）:")
        for s in bad_examples:
            print("  -", s)

    # 过滤并返回
    clean_df = df.loc[valid_mask].reset_index(drop=True)
    print(f"删除后行数: {len(clean_df)}  | 已删除: {n_invalid}")
    return clean_df

In [34]:
# 仅对训练集删除无效 SMILES（测试集不要删）
train_clean = drop_invalid_smiles(train_df, smiles_col="SMILES")

# 如果想看看测试集的无效比例，但不删除：
_ = drop_invalid_smiles(test, smiles_col="SMILES", show_examples=3)


[SMILES 校验] 总计: 10910  | 无效: 6 (0.055%)
无效样例（最多展示几条）:
  - *C(F)(F)CC(F)([R])C(*)(F)F
  - *CN([R'])Cc2cc([R]c1cc(*)c(O)c(CN([R'])C*)c1)cc(*)c2O
  - *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
  - *OC2OC(CO[R])C(OC1OC(CO[R])C(*)C(O[R])C1O[R])C(O[R])C2O[R]
  - *O[Si](*)([R])[R]
删除后行数: 10904  | 已删除: 6
[SMILES 校验] 总计: 3  | 无效: 0 (0.000%)
删除后行数: 3  | 已删除: 0


In [48]:
def rdkit_descriptors_from_list(smiles_series: pd.Series, desc_names: list, on_fail=np.nan) -> pd.DataFrame:
    """
    只支持 rdkit.Chem.Descriptors.descList 中的名字。
    - 未知名称会直接报错（帮助你发现拼写问题）
    - 解析失败的 SMILES 行填 on_fail（默认 NaN）
    返回：与输入索引对齐的 DataFrame
    """
    # descList 是 (name, func) 的列表
    avail = dict(Descriptors.descList)
    bad = [n for n in desc_names if n not in avail]
    if bad:
        raise ValueError(f"Unknown descriptor names (not in Descriptors.descList): {bad}")

    out_rows = []
    for smi in smiles_series.astype(str).tolist():
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            out_rows.append({n: on_fail for n in desc_names})
            continue
        row = {}
        for n in desc_names:
            try:
                row[n] = float(avail[n](mol))
            except Exception:
                row[n] = on_fail
        out_rows.append(row)

    return pd.DataFrame(out_rows, index=smiles_series.index)

In [55]:
from tqdm import tqdm

In [58]:
# ===== 1) 计算 RDKit 全量官方描述符（不含手工字段） =====
def compute_all_rdkit_descriptors(smiles_series: pd.Series) -> pd.DataFrame:
    names, funcs = zip(*Descriptors.descList)          # 官方名称 + 计算函数
    rows = []
    for s in smiles_series.astype(str):
        mol = Chem.MolFromSmiles(s)
        if mol is None:
            rows.append([np.nan]*len(funcs))
            continue
        vals = []
        for f in funcs:
            try:
                vals.append(float(f(mol)))
            except Exception:
                vals.append(np.nan)
        rows.append(vals)
    return pd.DataFrame(rows, columns=list(names), index=smiles_series.index)

# ===== 2) 别名映射，确保 filters 里的名字能对上 RDKit 列 =====
ALIAS = {
    'LogP': 'MolLogP',
    'RotatableBonds': 'NumRotatableBonds',
    'HBA': 'NumHAcceptors',
    'HBD': 'NumHDonors',
    'NumHetero': 'NumHeteroatoms',
    'RingCount': 'NumRings',        # 你 filters 里是 RingCount，RDKit 通常叫 NumRings
}

def map_and_select(filter_names: list[str], desc_df: pd.DataFrame) -> list[str]:
    mapped = [ALIAS.get(n, n) for n in filter_names]
    keep = [n for n in mapped if n in desc_df.columns]
    missing = [n for n in mapped if n not in desc_df.columns]
    if missing:
        print(f"[warn] dropped (not in RDKit): {missing[:12]}{' ...' if len(missing)>12 else ''}")
    return keep

# ===== 3) 组装某个目标的训练/测试特征矩阵（可拼接你的字符级特征列） =====
def build_Xy_for_target(
    df: pd.DataFrame,               # 含 SMILES 与各目标的总表（已去掉无效 SMILES）
    desc_df: pd.DataFrame,          # 与 df 行对齐的 RDKit 描述符全表
    target: str,                    # 'Tg' / 'FFV' / ...
    filters: dict,                  # 你给的 filters 字典
    extra_feat_cols: list[str] | None = None,   # 可选：你的字符级/其它特征列名
    smiles_col: str = 'SMILES'
):
    # 选择该目标要用的 RDKit 列
    rd_cols = map_and_select(filters[target], desc_df)
    parts = [desc_df[rd_cols]]
    if extra_feat_cols:
        # 只取数值列 & 现存列
        extra = [c for c in extra_feat_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
        if extra:
            parts.append(df[extra])
    X_full = pd.concat(parts, axis=1)

    # 仅保留该目标有标签的行
    rows = df[target].notna()
    X = X_full.loc[rows].copy()
    y = df.loc[rows, target].astype(float).copy()
    return X, y, list(X.columns)

# ===== 4) 简单的中位数填充（只填特征，不动 target） =====
def median_impute(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    for c in X.columns:
        if X[c].isna().any():
            X[c] = X[c].fillna(X[c].median())
    return X

TypeError: unsupported operand type(s) for |: 'types.GenericAlias' and 'NoneType'

In [54]:
# 先一次性算好 RDKit 全量描述符
desc_train = compute_all_rdkit_descriptors(train_clean['SMILES'])

NameError: name 'test_clean' is not defined

In [ ]:
X_tg, y_tg, tg_cols = build_Xy_for_target(
    df=train_clean,
    desc_df=desc_train,
    target='Tg',
    filters=filters,
    extra_feat_cols=char_feat_cols
)
X_tg = median_impute(X_tg)                 # 只填特征
print("Tg features:", len(tg_cols), X_tg.shape, y_tg.shape)

# 测试集用相同列
# 注意：desc_test 只有描述符，没有 target；extra 特征直接从 test_clean 取
X_tg_test = pd.concat(
    [
        desc_test[[c for c in tg_cols if c in desc_test.columns]],
        test_clean[[c for c in tg_cols if c in test_clean.columns]]  # 这行通常为空；仅当 tg_cols 里包含了你的字符列名时才会取到
    ],
    axis=1
)
# 对齐训练列次序（有的列在测试缺失，用 NaN 占位后再中位数填充）
X_tg_test = X_tg_test.reindex(columns=tg_cols)
X_tg_test = median_impute(X_tg_test)
print("Tg test matrix:", X_tg_test.shape)